In [ ]:

%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import tqdm
import tqdm.contrib.concurrent
import re
from collections import namedtuple
import ipywidgets
import math
from IPython.display import display
from scipy import special, optimize
from typing import Callable
import itertools

from experimental.beacon_sim.rollout_statistics_pb2 import AllStatistics, RolloutStatistics
from experimental.beacon_sim.beacon_sim_debug_pb2 import BeaconSimDebug
from experimental.beacon_sim import plot_trials
from experimental.beacon_sim import correlated_beacons_python as cb


from sophus.se2 import Se2 as SE2
from sophus.so2 import So2 as SO2
from sophus.complex import Complex as Complex
from sophus.matrix import Vector2

np.set_printoptions(linewidth=200)

import importlib
importlib.reload(plot_trials)
importlib.reload(cb)

In [ ]:
data_dir = os.path.expanduser("/home/erick/scratch/correlated_investigation/")

In [ ]:
Trial = namedtuple('Trial', ['trial_idx', 'cov_size', 'trace'])
Instance = namedtuple('Instance', ['instance_idx', 'start_position', 'goal_position', 'path_length', 'plan', 'trials'])
trace_finder = re.compile(r'.*?([0-9]+)/([0-9]+)\.pb')
instance_finder = re.compile(r'.*?([0-9]+)\.pb')
State = namedtuple('State', ['time_of_validity', 'local_from_true_robot', 'local_from_est_robot', 'robot_cov'])


def unpack_debug_msg(debug: BeaconSimDebug):
    complex_rot = debug.local_from_true_robot.rotation.complex 
    local_from_true_robot = SE2(
        SO2(Complex(complex_rot.real, complex_rot.imag)),
        plot_trials.matrix_from_proto(debug.local_from_true_robot.translation)
    )
    
    ekf_mean = SE2.exp(plot_trials.matrix_from_proto(debug.posterior.mean)[:3].squeeze().tolist())
    ekf_cov = plot_trials.matrix_from_proto(debug.posterior.cov)[:3, :3]
    
    local_from_est_robot = ekf_mean
    
    
    return State(
        time_of_validity=debug.time_of_validity.ticks_since_epoch/1e9,
        local_from_true_robot=local_from_true_robot,
        local_from_est_robot=local_from_est_robot,
        robot_cov = ekf_cov,
    )
    

def unpack_plan(entire_plan: list[BeaconSimDebug]):
    return [unpack_debug_msg(msg) for msg in entire_plan]

def unpack_trials(trials: list[RolloutStatistics]):
    return [Trial(trial_idx=stat.trial_idx, cov_size=plot_trials.compute_covariance_size(stat), trace=unpack_plan(stat.entire_plan)) for i, stat in enumerate(trials)]

def compute_path_length(stats: AllStatistics):
    current_pos = tuple(stats.local_from_start.translation.data)
    dist_m = 0.0
    for path_idx in stats.plan[1:]:
        if path_idx < 0:
            new_pos = (stats.goal.x, stats.goal.y)
        else:
            new_pos = tuple(stats.road_map.points[path_idx].data)
        
        edge_dist_m = ((current_pos[0] - new_pos[0]) ** 2 + (current_pos[1] - new_pos[1]) ** 2) ** 0.5
        dist_m += edge_dist_m
        current_pos = new_pos
    return dist_m

def load_instance(file_name: str) -> Instance:
    with open(file_name, 'rb') as file_in:
        stats = AllStatistics()
        stats.ParseFromString(file_in.read())
    
    m = trace_finder.search(file_name)
    if m:
        instance_idx = int(m.group(1))
    else:
        m = instance_finder.search(file_name)
        instance_idx = int(m.group(1))
    
    
    
    return Instance(instance_idx=instance_idx,
                    start_position=list(stats.local_from_start.translation.data),
                    goal_position=(stats.goal.x, stats.goal.y),
                    path_length = compute_path_length(stats),
                    plan = list(stats.plan),
                    trials = unpack_trials(stats.statistics))


In [ ]:
def load_road_map_and_world_map(file_path):
    with open(file_path, 'rb') as file_in:
        stats = AllStatistics()
        stats.ParseFromString(file_in.read())
        
    return stats.road_map, stats.world_map_config

In [ ]:
file_paths = [os.path.join(data_dir, file_name) for file_name in os.listdir(data_dir) if file_name.endswith('.pb')]

In [ ]:
road_map, world_map = load_road_map_and_world_map(file_paths[0])

In [ ]:
instances = tqdm.contrib.concurrent.process_map(load_instance, file_paths, chunksize=1, )
instances = sorted(instances, key=lambda x: x.instance_idx)

In [ ]:
def missing_from_configuration(configuration, world_map):
    idxs = []
    for i in range(len(world_map.correlated_beacons.beacons)):
        if configuration & (1 << i) == 0:
            idxs.append(world_map.correlated_beacons.beacons[i].id)
    return sorted(idxs)

def plot_environment(road_map, world_map, plan, start_pos, goal_pos, configuration=None):
    if configuration is None:
        configuration = 0
    missing_beacons = missing_from_configuration(configuration, world_map)
        
    # plot the road map
    # Plot the edges
    line_segments = []
    for r in range(road_map.adj.num_rows):
        for c in range(r+1, road_map.adj.num_cols):
            idx = r * road_map.adj.num_cols + c
            if (road_map.adj.data[idx] == 1):
                line_segments.append([tuple(road_map.points[r].data), tuple(road_map.points[c].data)])
    edges = mpl.collections.LineCollection(line_segments, colors=(0.8, 0.8, 0.6, 1.0))
    ax = plt.gca()
    ax.add_collection(edges)
    
    # Plot the plan
    plan_points = []
    for node_idx in plan:
        if node_idx == -1:
            plan_points.append(start_pos)
        elif node_idx == -2:
            plan_points.append(goal_pos)
        else:
            plan_points.append(tuple(road_map.points[node_idx].data))
    plan_x = [pt[0] for pt in plan_points]
    plan_y = [pt[1] for pt in plan_points]
    plt.plot(plan_x, plan_y, 'b')
    
    # Plot the start
    plt.plot(*start_pos, 'md', markersize=10)
    
    # Plot the goal
    plt.plot(*goal_pos, 'g*', markersize=15)
    
    # Plot the nodes
    rm_x = [pt.data[0] for pt in road_map.points]
    rm_y = [pt.data[1] for pt in road_map.points]
    plt.plot(rm_x, rm_y, 'rs')
    
    TEXT_X_OFFSET = 0.2
    TEXT_Y_OFFSET = 0.2
#     for i, pt in enumerate(road_map.points):
#         plt.text(pt.data[0] + TEXT_X_OFFSET, pt.data[1] + TEXT_Y_OFFSET, i)
        
    # Plot the beacons
    beacon_xs = [beacon.pos_x_m for beacon in world_map.correlated_beacons.beacons if beacon.id not in missing_beacons]
    beacon_ys = [beacon.pos_y_m for beacon in world_map.correlated_beacons.beacons if beacon.id not in missing_beacons]
    beacon_ids = [beacon.id for beacon in world_map.correlated_beacons.beacons if beacon.id not in missing_beacons]
    
    for x, y, beacon_id in zip(beacon_xs, beacon_ys, beacon_ids):
        plt.text(x + TEXT_X_OFFSET, y + TEXT_Y_OFFSET, beacon_id)
    
    plt.plot(beacon_xs, beacon_ys, 'b^')
    plt.axis('equal')
    
def plot_covariances(trials, current_trial):
    sorted_trials = sorted(trials, key=lambda trial: trial.cov_size)
    plt.plot([t.cov_size for t in sorted_trials], list(range(len(sorted_trials))))
    sorted_idx = sorted_trials.index(current_trial)
    plt.plot(current_trial.cov_size, sorted_idx, 'g*', markersize=15)
    
def make_plots(fig, trial_idx, instance, trials, road_map, world_map):
    trial = trials[trial_idx]
    plt.figure(fig.number)
    plt.gcf().clear()
    plt.subplot(121)
    plot_environment(
        road_map, world_map, instance.plan, instance.start_position,
        instance.goal_position, configuration=trial.trial_idx)
    plt.title(f'Instance: {instance.instance_idx} Trial: {trial.trial_idx}, Path Length: {instance.path_length: 0.2f}')
    plt.xlabel('X (m)')
    plt.ylabel('Y (m)')
    plt.subplot(122)
    plot_covariances(trials, trial)
    plt.xlabel('$|\Sigma|$')
    plt.ylabel('Count')
    plt.title('CDF of $|\Sigma|$')
    plt.tight_layout()
    
def plot_results(road_map, world_map, instances):
    path_length_order_checkbox = ipywidgets.Checkbox(value=True, description='Sorted Instances?')
    instance_slider = ipywidgets.IntSlider(min = 0, max=len(instances)-1, step=1, description='Instance Idx', value=0)
    sorted_trial_checkbox = ipywidgets.Checkbox(value=True, description = "Sorted Trials?")
    fig = plt.figure(figsize=(12, 6))
    
    hbox = ipywidgets.HBox()
    
    sorted_instances = sorted(instances, key=lambda x: x.path_length)
    
    def on_instance_change(change):
        if on_instance_change.trial_slider:
            on_instance_change.trial_slider.close()
            hbox.children = hbox.children[:-1]
        
        instance_idx = instance_slider.value
        should_use_sorted_path_length = path_length_order_checkbox.value
        instances_to_use = sorted_instances if should_use_sorted_path_length else instances
        instance = instances_to_use[instance_idx]
        
        trials = instance.trials
        sorted_trials = sorted(trials, key=lambda trial: trial.cov_size)
        style = {"description_width": "initial"}
        on_instance_change.trial_slider = ipywidgets.IntSlider(
            min=0, max=len(instance.trials)-1, step=1,
            description=f'Trial Idx for instance {instance.instance_idx}', value=0, style=style)
        
        def on_trial_change(change):
            trial_idx = change["owner"].value
            should_use_sorted_trial = sorted_trial_checkbox.value
            trials_to_use = sorted_trials if should_use_sorted_trial else trials
            make_plots(fig, trial_idx, instance, trials_to_use, road_map, world_map)
            
        on_instance_change.trial_slider.observe(on_trial_change, 'value')
        on_trial_change({'owner': on_instance_change.trial_slider})
        hbox.children = (*hbox.children, on_instance_change.trial_slider)
    on_instance_change.trial_slider = None
    
    def on_sorted_change(change):
        instance_idx = instance_slider.value
        trial_idx = on_instance_change.trial_slider.value
        should_use_sorted_trial = sorted_trial_checkbox.value
        should_use_sorted_path_length = path_length_order_checkbox.value
        
        instances_to_use = sorted_instances if should_use_sorted_path_length else instances
        instance = instances_to_use[instance_idx]
        
        trials = instance.trials
        sorted_trials = sorted(trials, key=lambda trial: trial.cov_size)
        trials_to_use = sorted_trials if should_use_sorted_trial else trials
        
        make_plots(fig, trial_idx, instance, trials_to_use, road_map, world_map)
        
    
    instance_slider.observe(on_instance_change, 'value')
    path_length_order_checkbox.observe(on_sorted_change, 'value')
    sorted_trial_checkbox.observe(on_sorted_change, 'value')
    hbox.children = (path_length_order_checkbox, instance_slider, sorted_trial_checkbox)
    display(hbox)
    on_instance_change({'owner': instance_slider})
    
    

In [ ]:
plot_results(road_map, world_map, instances)

In [ ]:
print(instances[0].trials[0])
print(instances[0].trials[16])

In [ ]:
trial_0 = load_instance('/home/erick/scratch/correlated_investigation/trial_000017/0.pb')
trial_16 = load_instance('/home/erick/scratch/correlated_investigation/trial_000017/16.pb')

In [ ]:
trial_0

In [ ]:


def plot_trace(instance, fig):
    # Plot axes
    AXIS_LENGTH = 0.05
    trace = instance.trials[0].trace
    locs_in_local = np.array([s.local_from_true_robot * Vector2(0.0, 0.0) for s in trace]).transpose(0,2,1)
    true_x_tips_in_local = np.array([s.local_from_true_robot * Vector2(AXIS_LENGTH, 0.0) for s in trace]).transpose(0,2,1)
    true_y_tips_in_local = np.array([s.local_from_true_robot * Vector2(0.0, AXIS_LENGTH) for s in trace]).transpose(0,2,1)
    
    x_axes_in_local = np.concatenate([locs_in_local, true_x_tips_in_local], axis=1)
    y_axes_in_local = np.concatenate([locs_in_local, true_y_tips_in_local], axis=1)
    
    # plot axes
    ax = fig.subplots()
    ax.add_collection(mpl.collections.LineCollection(x_axes_in_local, colors='r'))
    ax.add_collection(mpl.collections.LineCollection(y_axes_in_local, colors='g'))
    
    
    # Plot Cov Matrices
    for s in trace:
        NUM_PTS = 20
        theta = np.linspace(0, 2*np.pi, NUM_PTS)
        COV_SIGMA = 1.0
        pts = COV_SIGMA * np.stack([np.cos(theta), np.sin(theta), np.zeros_like(theta)])

        cov_sqrt = np.linalg.cholesky(s.robot_cov)
        tangent_in_robot = cov_sqrt @ pts
        robot_from_ellipse_pts = [SE2(SO2(Complex(1.0, 0.0)), Vector2(*tangent_in_robot[:2, i].tolist())) for i in range(NUM_PTS)]
        local_from_ellipse_pts = np.array([(s.local_from_true_robot * r_from_e_pt) * Vector2(0.0,0.0) for r_from_e_pt in robot_from_ellipse_pts]).transpose(2,0,1)
        
        ax.add_collection(mpl.collections.LineCollection(local_from_ellipse_pts, colors='b'))
        
    
    ax.axis('equal')
    print(robot_from_ellipse_pts[4])
    
    plt.title(f'Instance {instance.instance_idx} Trial {instance.trials[0].trial_idx}')

In [ ]:
# [segment_id, pt_id, xy]
fig = plt.figure(figsize=(10, 6))
plot_trace(trial_0, fig)
plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(10, 6))
plot_trace(trial_16, fig)
plt.tight_layout()

In [ ]:

def independent_beacons(marginal_prob):
    ids = [beacon.id for beacon in world_map.correlated_beacons.beacons]
    clique = cb.BeaconClique(p_beacon=marginal_prob, p_no_beacons=(1-marginal_prob) ** len(ids), members=ids)
    return cb.create_correlated_beacons(clique)

def all_correlated(marginal_prob, p_no_beacon):
    ids = [beacon.id for beacon in world_map.correlated_beacons.beacons]
    clique = cb.BeaconClique(p_beacon=marginal_prob, p_no_beacons=p_no_beacon, members=ids)
    return cb.create_correlated_beacons(clique)

def clustered_beacons(marginal_prob, p_no_beacon, clusters):
    out = None
    for members in clusters:
        clique = cb.BeaconClique(p_beacon=marginal_prob, p_no_beacons=p_no_beacon**(1.0/len(clusters)), members=members)
        if out is None:
            out = cb.create_correlated_beacons(clique)
        else:
            out *= cb.create_correlated_beacons(clique)
    
    return out


def corner_beacons(marginal_prob, p_no_beacon):
    clusters = [(51, 5, 53), (1, 3, 54), (50, 4, 52), (0, 2, 55)]
    return clustered_beacons(marginal_prob, p_no_beacon, clusters)

def sides_beacons(marginal_prob, p_no_beacon):
    clusters = [(0, 2, 4), (50, 52, 54), (1, 3, 5), (51, 53, 55)]
    return clustered_beacons(marginal_prob, p_no_beacon, clusters)

def cycle_beacons(marginal_prob, p_no_beacon):
    clusters = [(0, 50, 1, 51), (2, 52, 3, 53), (4, 54, 5, 55)]
    return clustered_beacons(marginal_prob, p_no_beacon, clusters)

def compute_expected_covariance_size(beacon_pot, world_map, trials):
    expected_cov_size = 0.0
    for trial in trials:
        missing_beacons = missing_from_configuration(trial.trial_idx, world_map)
        beacon_presence = {beacon.id: not (beacon.id in missing_beacons) for beacon in world_map.correlated_beacons.beacons}
        expected_cov_size += np.exp(beacon_pot.log_prob(beacon_presence)) * trial.cov_size
    return expected_cov_size

# Correlation types
# all independent
# sides
# corners
# cycling

In [ ]:
marginal_prob = 0.4
p_no_beacons = 0.1
distributions = {
    'independent': independent_beacons(marginal_prob),
    'all_correlated': all_correlated(marginal_prob, p_no_beacons),
    'corners': corner_beacons(marginal_prob, p_no_beacons),
    'sides': sides_beacons(marginal_prob, p_no_beacons),
    'cycle': cycle_beacons(marginal_prob, p_no_beacons),    
}


In [ ]:
p_no_beacon_list = np.linspace(0.58, 0.02, 5)
dist_from_p_no_beacon = {}
marginal_prob = 0.4
for p_no_beacon in p_no_beacon_list:
    dist_from_p_no_beacon[p_no_beacon] = all_correlated(marginal_prob, p_no_beacon)

In [ ]:
def pot_expectation(potential: cb.BeaconPotential, f: Callable[dict[int, bool], float]):
    out = 0.0
    for values in itertools.product([False, True], repeat=len(potential.members)):
        assignments = {m: v for m, v in zip(potential.members, values)}
        p = np.exp(potential.log_prob(assignments))
        value = f(assignments)
        out += p * value
    return out

def pot_entropy(potential: cb.BeaconPotential):
    return -pot_expectation(potential, potential.log_prob)
        
        
        
    

In [ ]:
for p_no_beacon, dist in dist_from_p_no_beacon.items():
    print(p_no_beacon, pot_entropy(dist))

In [ ]:
expected_cov_size = {key: [] for key in dist_from_p_no_beacon}

def compute_cov_size_for_instance(instance):
    out = {'instance_idx': instance.instance_idx}
    for key, dist in dist_from_p_no_beacon.items():
        out[key] = compute_expected_covariance_size(dist, world_map, instance.trials)
    return out
        
results = tqdm.contrib.concurrent.process_map(compute_cov_size_for_instance, instances, chunksize=1)

In [ ]:

for key in expected_cov_size:
    covs = sorted([stats[key] for stats in results])
    plt.figure('cdf')  
    plt.plot(covs, list(range(len(covs))), label=f'$P(0)=${key:0.2f}')
    
    plt.figure('pdf')
    plt.hist(covs, label=f'$P(0)=${key:0.2f}', histtype='step', bins=list(range(0, 250, 5)), linewidth=1)

plt.figure('cdf')
plt.xlabel('$\mathbb{E}[|\Sigma|]$')
plt.ylabel('Instance Counts')
plt.title('Covariance Size vs P(No Beacons) CDF')
plt.legend(title='$P_i(b_i) = 0.4$')
plt.tight_layout()

plt.figure('pdf')
plt.legend(title='$P_i(b_i) = 0.4$')
plt.xlabel('$\mathbb{E}[|\Sigma|]$')
plt.ylabel('Instance Counts')
plt.title('Covariance Size vs P(No Beacons) PDF')
plt.tight_layout()

